In [1]:
import requests
import pandas as pd
import json
import time

#  Ingesting Data Using An API

In This our goal is to extract the available data stock news on polygon.ai

<br> Step 1:  we will look for the endpoint -api (https://polygon.io/docs/stocks/get_v2_reference_news) that provides the data we need. In the case of news extraction, it appears that the GET /v3/reference/news endpoint fits our task.

<br> Step 2: we are defining the parameters which are required for the below extraction function. However, due to the 1,000 ticker limit per page, we are using the next_url response attribute to obtain the next page of information

In [2]:
from datetime import datetime, timedelta
yesterday = datetime.now() - timedelta(1)
yest_date = datetime.strftime(yesterday, '%Y-%m-%d')


In [3]:
def return_commonstock_tickers(token, initial_load=False):
    url = 'https://api.polygon.io/v2/reference/news'
#     ticker=AAPL&published_utc.lte=2022-07-01&apiKey=Oqpqg_khSPFkn0cXejujjGR9VBDp13bu
    # taking yesterdays date for using in our inc load 
    yesterday = datetime.now() - timedelta(1)
    yest_date = datetime.strftime(yesterday, '%Y-%m-%d')

    published_utc ="1900-01-01" if  initial_load else yest_date
    print("published_utc::", published_utc)
    parameters = {
        'apiKey': token, # your API key
        'ticker': 'AAPL', # query common stocks
        'published_utc.gte': published_utc,
        'limit': 500 # extract max data possible
    }

    try:
        tickers_json = requests.get(url, parameters).json()
        tickers_list = tickers_json['results']
        if 'next_url' not in tickers_json.keys():
            return tickers_list
#         print(tickers_json['next_url'])
        next_url = tickers_json['next_url'] if tickers_json['next_url'] else None
#         print(next_url)
        while next_url:
            tickers_json = requests.get(tickers_json["next_url"], parameters).json()
            tickers_list.extend(tickers_json["results"])
            if 'next_url' not in tickers_json.keys():
                break
            # trigger a wait since free tier is limited to 5 calls/min
            time.sleep(12)
            
    except Exception as e:
        print(e)
        return None
    
    return tickers_list

#  Write Data into CSV or Text file

<br> here we are first calling our extraction function which is created above and initialize a variable with the returned data. To write out as a CSV file, the response must be converted into a Pandas DataFrame then utilize the .to_csv() method.

<br> Apart from our API Token we are also passing True or flase flag. True indicates for the full load of the data where are False flag indicates the inc load 

In [4]:
token = 'Oqpqg_khSPFkn0cXejujjGR9VBDp13bu'

#True for initial load and False for inc loads
tickers = return_commonstock_tickers(token, False)

print(tickers)
# write out as csv
df_cs_tickers = pd.DataFrame(tickers)
df_cs_tickers.to_csv('polygon_stock_news.csv', index = False)

# # write out as txt
# with open('polygon_cs_tickers_1.txt', 'w') as outfile:
#     json.dump(tickers, outfile, indent=4)

published_utc:: 2022-08-06
[{'id': 'UcXHjYE5-vk8i7OTlQV3blTmyDeMRM5t3rzL0YbO2ig', 'publisher': {'name': 'The Motley Fool', 'homepage_url': 'https://www.fool.com/', 'logo_url': 'https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg', 'favicon_url': 'https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico'}, 'title': "Should You Really Follow Warren Buffett's Lead on Apple Stock?", 'author': 'newsfeedback@fool.com (Rich Duprey)', 'published_utc': '2022-08-07T10:15:00Z', 'article_url': 'https://www.fool.com/investing/2022/08/07/should-you-really-follow-warren-buffetts-lead-on-a/', 'tickers': ['AAPL', 'BRK.A', 'BRK.B'], 'image_url': 'https://g.foolcdn.com/editorial/images/693552/warren-buffett-motley-fool.jpg', 'description': 'The Oracle of Omaha has made a huge bet on the tech giant.', 'keywords': ['investing']}, {'id': '7IgvYE0FG0pGcpEGVukLh0mo4UQPQsYiCYKuiCsFJMc', 'publisher': {'name': 'The Motley Fool', 'homepage_url': 'https://www.fool.com/', 'logo_url': 'https:

In [5]:
data = pd.read_csv("polygon_stock_news.csv")
data.head() 

,id,publisher,title,author,published_utc,article_url,tickers,image_url,description,keywords
0,UcXHjYE5-vk8i7OTlQV3blTmyDeMRM5t3rzL0YbO2ig,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Should You Really Follow Warren Buffett's Lead...,newsfeedback@fool.com (Rich Duprey),2022-08-07T10:15:00Z,https://www.fool.com/investing/2022/08/07/shou...,"['AAPL', 'BRK.A', 'BRK.B']",https://g.foolcdn.com/editorial/images/693552/...,The Oracle of Omaha has made a huge bet on the...,['investing']
1,7IgvYE0FG0pGcpEGVukLh0mo4UQPQsYiCYKuiCsFJMc,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Buying Cheap Stocks Isn't the Same as Value In...,newsfeedback@fool.com (Stefon Walters),2022-08-07T10:00:00Z,https://www.fool.com/investing/2022/08/07/buyi...,"['GOOGL', 'AAPL', 'TSLA', 'BAC', 'WFC', 'XOM',...",https://g.foolcdn.com/editorial/images/693329/...,Don't let the price fool you.,['investing']
2,j9YmZJOwT2bjdOw4p1vhJwr1EjzvH-kgitQTSSpzabg,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Could Apple Join Forces With Bitcoin?,newsfeedback@fool.com (RJ Fulton),2022-08-06T22:11:00Z,https://www.fool.com/investing/2022/08/06/coul...,"['AAPL', 'GS', 'MA', 'PYPL', 'SQ']",https://g.foolcdn.com/editorial/images/693637/...,Why an integration with Bitcoin could be next ...,['investing']
3,WvCCi79iC5w3kO4LWuuvvEIrRF57oKPKIEw2St5NBFI,"{'name': 'The Motley Fool', 'homepage_url': 'h...","Snap's Struggles Set It Apart, and Streaming ""...",newsfeedback@fool.com (Chris Hill),2022-08-06T15:30:00Z,https://www.fool.com/investing/2022/08/06/snap...,"['AMZN', 'SNAP', 'BRK.A', 'BRK.B', 'AAPL', 'MS...",https://g.foolcdn.com/editorial/images/691768/...,"Snap stock fell nearly 40% on Friday, taking b...",['investing']
4,3QopZQ_i_3E-5E-ROgYyeD_omBPDFBQyJMa38pydknE,"{'name': 'The Motley Fool', 'homepage_url': 'h...",The Streaming Wars Heat Up With NFL+ Launch; W...,newsfeedback@fool.com (Dani Cook),2022-08-06T14:30:00Z,https://www.fool.com/investing/2022/08/06/the-...,"['AMZN', 'AAPL', 'DIS']",https://g.foolcdn.com/editorial/images/693097/...,Streaming giants might not have many sports li...,['investing']


In [6]:
news_df = data[["id","title","author","published_utc", "keywords"]].copy()
news_df.head()

,id,title,author,published_utc,keywords
0,UcXHjYE5-vk8i7OTlQV3blTmyDeMRM5t3rzL0YbO2ig,Should You Really Follow Warren Buffett's Lead...,newsfeedback@fool.com (Rich Duprey),2022-08-07T10:15:00Z,['investing']
1,7IgvYE0FG0pGcpEGVukLh0mo4UQPQsYiCYKuiCsFJMc,Buying Cheap Stocks Isn't the Same as Value In...,newsfeedback@fool.com (Stefon Walters),2022-08-07T10:00:00Z,['investing']
2,j9YmZJOwT2bjdOw4p1vhJwr1EjzvH-kgitQTSSpzabg,Could Apple Join Forces With Bitcoin?,newsfeedback@fool.com (RJ Fulton),2022-08-06T22:11:00Z,['investing']
3,WvCCi79iC5w3kO4LWuuvvEIrRF57oKPKIEw2St5NBFI,"Snap's Struggles Set It Apart, and Streaming ""...",newsfeedback@fool.com (Chris Hill),2022-08-06T15:30:00Z,['investing']
4,3QopZQ_i_3E-5E-ROgYyeD_omBPDFBQyJMa38pydknE,The Streaming Wars Heat Up With NFL+ Launch; W...,newsfeedback@fool.com (Dani Cook),2022-08-06T14:30:00Z,['investing']


In [7]:
len(news_df)

10

In [8]:
news_df = news_df.fillna("NoData")

In [9]:
Id_list ="','".join(news_df['id'])
#"select '" + Id_list + '\'from'
#type(Id_list)

In [10]:
# "DELETE FROM stock_news WHERE id in ('"+Id_list+"')"

In [11]:
# mydata.head()

In [12]:
# data.info()

# Loading the ingested data into MySql server 

<br> Here we are conneting to local mysql server giving required parameters as below and then creating the database

In [13]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', user='root',  
                        password='Qwerty@123')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS stock")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Database is created


In [14]:
news_df.columns

Index(['id', 'title', 'author', 'published_utc', 'keywords'], dtype='object')

<br> In this section we are creating the structure of the table with the required attributes and doing delete and insert operation 

In [15]:
# import mysql.connector as msql
# from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', database='stock', user='root', password='Qwerty@123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS stock_news;')
        print('Creating table....')
# in the below line please pass the create table statement which you want #to create
        cursor.execute("CREATE TABLE stock_news(id varchar(255),title varchar(1000),author varchar(255),published_utc varchar(255),keywords varchar(1000))")
        print("Table is created....")
        #loop through the data frame
        sql1 = "DELETE FROM stock_news WHERE id in ('"+Id_list+"')"
        #print(sql1)
        cursor.execute(sql1)
        conn.commit()
        for i,row in news_df.iterrows():
            #here %S means string values 
            
            #print("No of records deleted: ", len())
            sql2 = "INSERT INTO stock.stock_news VALUES (%s,%s,%s,%s,%s)"
            cursor.execute(sql2, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)



You're connected to database:  ('stock',)
Creating table....
Table is created....
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [16]:
# Execute query
sql = "SELECT * FROM stock.stock_data"
cursor.execute(sql)
# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

('2', 'AAC', 'Ares Acquisition Corporation', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1829430.0, '0', '0', '2022-08-05T00:00:00Z')
('3', 'AACI', 'Armada Acquisition Corp. I Common Stock', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1844820.0, 'BBG011XR7306', 'BBG011XR7315', '2022-08-05T00:00:00Z')
('4', 'AADI', 'Aadi Bioscience, Inc. Common Stock', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1422140.0, 'BBG002WN7DT2', 'BBG002WN7DV9', '2022-08-05T00:00:00Z')
('5', 'AAIC', 'Arlington Asset Investment Corp.', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1209030.0, 'BBG000BD1373', 'BBG001S6DRN4', '2022-08-05T00:00:00Z')
('6', 'AAL', 'American Airlines Group Inc.', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 6201.0, 'BBG005P7Q881', 'BBG005P7Q907', '2022-08-05T00:00:00Z')
('7', 'AAMC', 'Altisource Asset Mgmt Corp', 'stocks', 'us', 'XASE', 'CS', 1, 'usd', 1555070.0, 'BBG003PNL136', 'BBG003PNL145', '2022-08-05T00:00:00Z')
('8', 'AAME', 'Atlantic American Corp', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 8177.0, 'B

('1872', 'EVOJ', 'Evo Acquisition Corp. Class A Common Stock', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1834340.0, 'BBG00Z0DX9R8', 'BBG00Z0DX9V3', '2022-08-05T00:00:00Z')
('1873', 'EVOK', 'Evoke Pharma, Inc.', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1403710.0, 'BBG0026ZG0X7', 'BBG0026ZG0Y6', '2022-08-05T00:00:00Z')
('1874', 'EVOP', 'EVO Payments, Inc. Class A Common Stock', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1704600.0, 'BBG00KRX3MG4', 'BBG00KRX3N72', '2022-08-05T00:00:00Z')
('1875', 'EVR', 'Evercore Inc.', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1360900.0, 'BBG000BP30Y0', 'BBG001SC3S41', '2022-08-05T00:00:00Z')
('1876', 'EVRG', 'Evergy, Inc.', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1711270.0, 'BBG00H433CR2', 'BBG00H433CS1', '2022-08-05T00:00:00Z')
('1877', 'EVRI', 'Everi Holdings Inc', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1318570.0, 'BBG000H1B407', 'BBG001SLY556', '2022-08-05T00:00:00Z')
('1878', 'EVTC', 'EVERTEC, INC.', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1559860.0, 'BBG00

('3761', 'NVST', 'Envista Holdings Corporation Common stock, $0.01 par value per share', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1757070.0, 'BBG00LN4B5N0', 'BBG00LN4B6D9', '2022-08-05T00:00:00Z')
('3762', 'NVT', 'nVent Electric plc Ordinary Shares', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1720640.0, '0', '0', '2022-08-05T00:00:00Z')
('3763', 'NVTA', 'Invitae Corporation', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1501130.0, 'BBG005DJFD43', 'BBG005DJFD52', '2022-08-05T00:00:00Z')
('3764', 'NVTS', 'Navitas Semiconductor Corporation Common Stock', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1821770.0, 'BBG00YQ6KTL4', 'BBG00YQ6KTM3', '2022-08-05T00:00:00Z')
('3765', 'NVVE', 'Nuvve Holding Corp. Common Stock', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1836880.0, 'BBG00Z49PKM2', 'BBG00Z49PKN1', '2022-08-05T00:00:00Z')
('3766', 'NWBI', 'Northwest Bancshares, Inc', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1471260.0, 'BBG000BG8NP5', 'BBG001S64KK2', '2022-08-05T00:00:00Z')
('3767', 'NWE', 'NorthWestern C

('5371', 'UP', 'Wheels Up Experience Inc.', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1819520.0, 'BBG00XTV85L4', 'BBG00XTV85M3', '2022-08-05T00:00:00Z')
('5372', 'UPC', 'Universe Pharmaceuticals Inc. Ordinary Shares', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1809620.0, '0', '0', '2022-08-05T00:00:00Z')
('5373', 'UPH', 'UpHealth, Inc.', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1770140.0, 'BBG00PC18QT1', 'BBG00PC18QW7', '2022-08-05T00:00:00Z')
('5374', 'UPLD', 'Upland Software, Inc.', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1505160.0, 'BBG005KC3JR5', 'BBG005KC3JS4', '2022-08-05T00:00:00Z')
('5375', 'UPS', 'United Parcel Service, Inc. Class B', 'stocks', 'us', 'XNYS', 'CS', 1, 'usd', 1090730.0, 'BBG000L9CV04', 'BBG001SDGSH1', '2022-08-05T00:00:00Z')
('5376', 'UPST', 'Upstart Holdings, Inc. Common stock', 'stocks', 'us', 'XNAS', 'CS', 1, 'usd', 1647640.0, 'BBG00PKCBY53', 'BBG00PKCBY62', '2022-08-05T00:00:00Z')
('5377', 'UPTD', 'TradeUP Acquisition Corp. Common Stock', 'stocks', 'us', 'XNAS', '